In [1]:
class treeNode():
    def __init__(self,feat,val,right,left):
        featureToSplitOn=feat
        valueOfSplit=val
        rightBranch=right
        leftBranch=left

In [2]:
from numpy import *

In [3]:
def loadDataSet(fileName):
    dataMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        curLine=line.strip().split('\t')
        fltLine=map(float,curLine)#map everything to float()
        dataMat.append(fltLine)
    return dataMat

In [4]:
def binSplitDataSet(dataSet,feature,value):# 根据feature的value，对dataSet进行划分
    mat0=dataSet[nonzero(dataSet[:,feature]>value)[0],:][0]
    mat1=dataSet[nonzero(dataSet[:,feature]<=value)[0],:][0]
    return mat0,mat1

In [5]:
'''
which generates the model for a leaf node. 
When chooseBestSplit() decides that you no longer should split the data, 
it will call regLeaf() to get a model for the leaf. 
The model in a regression tree is the mean value of the target variables.
'''
def regLeaf(dataSet):
    return mean(dataSet[:,-1])

In [6]:
#This function returns the squared error of the target variables in a given dataset
def regErr(dataSet):
    return var(dataSet[:,-1])*shape(dataSet)[0]#方差乘以个数

In [7]:
def chooseBestSplit(dataSet,leafType=regLeaf,errType=regErr,ops=(1,4)):
    #The variable tolS is a tolerance on the error reduction, and tolN is the minimum data instances to include in a split
    tolS=ops[0];tolN=ops[1]
    if len(set(dataSet[:,-1].T.tolist()[0]))==1:#check the number of unique values by creating a set from all the target variables
        return None,leafType(dataSet)
    m,n=shape(dataSet)
    S=errType(dataSet)
    bestS=inf;bestIndex=0;bestValue=0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:,featIndex]):
            mat0,mat1=binSplitDataSet(dataSet,featIndex,splitVal)
            if (shape(mat0)[0]<tolN) or (shape(mat1)[0]<tolN):
                continue
            newS=errType(mat0)+errType(mat1)
            if newS<bestS:
                bestIndex=featIndex
                bestValue=splitVal
                bestS=newS
    if (S-bestS)<tolS:#Exit if low error reduction
        return None,leafType(dataSet)
    mat0,mat1=binSplitDataSet(dataSet,bestIndex,bestValue)
    if (shape(mat0)[0]<tolN) or (shape(mat1)[0]<tolN):#Exit if split creates small dataset
        return None,leafType(dataSet)
    return bestIndex,bestValue

In [8]:
'''
The argument leafType is the function used to create a leaf. 
The argument errType is a function used for measuring the error on the dataset. 
The last argument, ops, is a tuple of parameters for creating a tree.
'''
def createTree(dataSet,leafType=regLeaf,errType=regErr,ops=(1,4)):
    feat,val=chooseBestSplit(dataSet,leafType,errType,ops)
    if feat==None:
        return val
    retTree={}
    retTree['spInd']=feat
    retTree['spVal']=val
    lSet,rSet=binSplitDataSet(dataSet,feat,val)
    retTree['left']=createTree(lSet,leafType,errType,ops)
    retTree['right']=createTree(rSet,leafType,errType,ops)
    return retTree
    

In [9]:
def isTree(obj):
    return (type(obj).__name__=='dict')

In [10]:
def getMean(tree):
    if isTree(tree['right']):
        tree['right']=getMean(tree['right'])
    if isTree(tree['left']):
        tree['left']=getMean(tree['left'])
    return (tree['left']+tree['right'])/2.0

In [11]:
def prune(tree,testDat):
    if shape(testData)[0]==0:
        return getMean(tree)
    if (isTree(tree['right'])) or (isTree(tree['left'])):
        lSet,rSet=binSplitDataSet(testData,tree['spInd'],tree['spVal'])
    if isTree(tree['left']):
        tree['left']=prune(tree['left'],lSet)
    if isTree(tree['right']):
        tree['right']=prune(tree['right'],rSet)
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet,rSet=binSplitDataSet(testData,tree['spInd'],tree['spVal'])
        errorNoMerge=sum(power(lSet[:,-1]-tree['left'],2))+sum(power(rSet[:,-1]-tree['right'],2))
        treeMean=(tree['left']+tree['right'])/2.0
        errorMerge=sum(power(testData[:,-1]-treeMean,2))
        if errorMerge<errorNoMerge:
            print "merging"
            return treeMean
        else:
            return tree
    else:
        return tree
    

In [12]:
def linearSolve(dataSet):
    m,n=shape(dataSet)
    X=mat(ones((m,n)));Y=mat(ones((m,1)))
    X[:,1:n]=dataSet[:,0:n-1];Y=dataSet[:,-1]
    xTx=X.T*X
    if linalg.det(xTx)==0.0:
        raise NameError('This matrix is singular,cannot do inverse,try increading the second value of ops')
    ws=xTx.T*(X.T*Y)
    return ws,X,Y

In [13]:
def modeLeaf(dataSet):
    ws,X,Y=linearSolve(dataSet)
    return ws

In [14]:
def modelErr(dataSet):
    ws,X,Y=linearSolve(dataSet)
    yHat=X*ws
    return sum(power(Y-yHat,2))

In [15]:
def regTreeEval(model,inDat):
    return float(model)

In [16]:
def modelTreeEval(model,inDat):
    n=shape(inDat)[1]
    X=mat(ones((1,n+1)))
    X[:,1:n+1]=inDat
    return float(X*model)

In [17]:
def treeForeCast(tree,inData,modelEval=regTreeEval):
    if not isTree(tree):
        return modelEval(tree,inData)
    if inData[tree['spInd']]>tree['spVal']:
        if isTree[tree['left']]:
            return treeForeCast(tree['left'],inData,modelEval)
        else:
            return modelEval(tree['left'],inData)
    else:
        if isTree(tree['right']):
            return treeForeCast(tree['right'],inData,modelEval)
        else:
            return modelEval(tree['right'],inData)

In [18]:
def createForeCast(tree,testData,modelEval=regTreeEval):
    m=len(testData)
    yHat=mat((m,1))
    for i in range(m):
        yHat[i,0]=treeForeCast(tree,mat(testData[i]),modelEval)
    return yHat